In [ ]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
# 0.5s

In [ ]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(3)
    c.close()
    db.close()
    return True

def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists DronaiVarle
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists DronaiVarle
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True
# 0.0s

In [11]:
# WebDriver
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/?p=1'
driver.get(url)
time.sleep(3)
source = driver.page_source
driver.close()
print(f'Prskaityta {url}')
# 18.0s

Prskaityta https://www.varle.lt/dronai/?p=1


In [ ]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div',{'class':'GRID_ITEM'})
for linkas in linkai:
    # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
    link = linkas.find('div', {'class':'product-title'}).find('a')
    # print(link['href'])
    add_link = 'https://www.varle.lt'+ link['href']
    # print(add_link)
    p_nuorodos.append(add_link)
print(len(p_nuorodos))
print(p_nuorodos)

pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)
print(pages_max_number)
# 0.6s

38
['https://www.varle.lt/dronai/dji-mini-4-pro-fly-more-combo-dji-rc-2-dronas--29184156.html', 'https://www.varle.lt/dronai/dronas-dji-mini-2-se--23792132.html', 'https://www.varle.lt/dronai/dronas-dji-mini-3-su-dji-rc-n1-valdymo-pultu--22802854.html', 'https://www.varle.lt/dronai/dronas-dji-mini-4k-fly-more-combo-su-dji-rc-n1c--38731147.html', 'https://www.varle.lt/dronai/avernus-dronas-s1s-4k-hd--37604985.html', 'https://www.varle.lt/dronai/avernus-dronas-l900-pro-gps--37604944.html', 'https://www.varle.lt/dronai/dronas-dji-mini-3-fly-more-combo-su-dji-rc-valdymo--21966035.html', 'https://www.varle.lt/dronai/dronas-dji-mavic-3-pro-cine-premium-combo-dji-rc--24575445.html', 'https://www.varle.lt/dronai/dronas-dji-mavic-3-pro-dji-rc--24505758.html', 'https://www.varle.lt/dronai/extra-digital-kroviklis-dji-mavic-mini-2-se--39704489.html', 'https://www.varle.lt/dronai/dji-avata-2-dronas--34746172.html', 'https://www.varle.lt/dronai/dji-neo-drone-fly-more-combo-dronas--38775265.html', 'h

In [ ]:
for nuoroda in p_nuorodos[:1]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = webdriver.Chrome(options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source
    driver.close()
    # 22.7s

https://www.varle.lt/dronai/dji-mini-4-pro-fly-more-combo-dji-rc-2-dronas--29184156.html


In [ ]:
bs = BeautifulSoup(source, 'html.parser')
pavadinimas = bs.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
print(pavadinimas)

# varle_kodas = bs.find('div', {'class':'main'}).find('span', {'itemprop':'sku'}).text
# print(varle_kodas)

kaina = bs.find('span', {'class':'price-value'}).text.strip()
print(kaina)

buvusi_kaina = bs.find('span', {'class':'previous-price'})
if buvusi_kaina != None:
    buvusi_kaina = buvusi_kaina.text.strip()
else:
    buvusi_kaina = None
print(buvusi_kaina)

specifiacija_keys = []
specifiacija_values = []
try:
    specifiacija = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
    for line in specifiacija:
        try:
            spec_key = line.find('span', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('span', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
        try:
            spec_key = line.find('div', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('div', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
except:
    pass

try:
    specifiacija_k = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
    for key in specifiacija_k:
        # print(key.text.strip())
        specifiacija_keys.append(key.text.strip())
    specifiacija_v = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
    for val in specifiacija_v:
        # print(val.text.strip())
        specifiacija_values.append(val.text.strip())       
except:
    pass

print(len(specifiacija_keys), specifiacija_keys)
print(len(specifiacija_values), specifiacija_values)

spec_filtrai_keys = []
spec_filtrai_values = []
try:
    spec_filtai = bs.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
    for line in spec_filtai:
        spec_key = line.find('span', {'class':'left-column'}).text.strip()
        # print(spec_key)
        spec_filtrai_keys.append(spec_key)
        spec_value = line.find('span', {'class':'right-column'}).text.strip()
        # print(spec_value)
        spec_filtrai_values.append(spec_value)
except:
    pass
    
print(len(spec_filtrai_keys), spec_filtrai_keys)
print(len(spec_filtrai_values), spec_filtrai_values)
# 0.4s

DJI Mini 4 Pro Fly More Combo ( DJI RC 2 ) dronas
1029.99
None
37 ['Gamintojas', 'Stabilizavimas', 'Spalva', 'Perdavimo atstumas (lauke ir be kliūčių)', 'Maksimalus kilimo greitis', 'Bendras megapikselių skaičius', 'Veikimo dažnis', 'Svoris (su baterija, g)', 'Didžiausias skrydžio aukštis', 'Baterijos pavadinimas', 'Talpa', 'Didžiausias skrydžio laikas', 'Vaizdo raiška', 'Sklandymas viduje', 'Maksimalus nusileidimo greitis', 'Jutiklio dydis', 'Įkrovimas', 'Maitinimo šaltinis', 'Tinkamos operacinės sitemos', 'Maksimalus greitis', 'Atminties kortelės tipas', 'Nuotolinis vardas', 'Filmų formatas', 'Bitų skaičius', 'Įrašymo laikmena', 'Matymo laukas', 'Jutiklio tipas', 'Veikimo temperatūra', 'Mobilioji programėlė', 'Maksimalus atsparumas vėjui', 'Nejudančio vaizdo failo tipas', 'Skraidyklės modelis', 'Iso', 'Baterijos tipas', 'Mobiliojo įrenginio laikiklis', 'Komplektacija', 'Modelio pavadinimas']
37 ['DJI', '3 ašių (3-axis)', 'pilka', 'CE: 10 000 m', '5 m/s', '48', '5,725 – 5,825 GHz', '2

In [9]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/?p=1'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)

counts = 1
psl = 1

while psl < pages_max_number + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []

    linkai = bs.find_all('div', {'class':'GRID_ITEM'})
    for linkas in linkai:
        # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
        link = linkas.find('div', {'class':'product-title'}).find('a')
        # print(link['href'])
        add_link = 'https://www.varle.lt'+ link['href']
        # print(add_link)
        p_nuorodos.append(add_link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        time.sleep(random.randint(5,10))
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='DronaiVarle', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs_preke = BeautifulSoup(source, 'html.parser')
        
        firstInfo = bs_preke.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
        duomenys['firstInfo'] = firstInfo
        # print(pavadinimas)


        kaina = bs_preke.find('span', {'class':'price-value'}).text.strip()
        duomenys['kaina'] = kaina
        # print(kaina)

        buvusi_kaina = bs_preke.find('span', {'class':'previous-price'})
        if buvusi_kaina != None:
            buvusi_kaina = buvusi_kaina.text.strip()
        else:
            buvusi_kaina = None
        duomenys['buvusiKaina'] = buvusi_kaina
        # print(buvusi_kaina)

        specifiacija_keys = []
        specifiacija_values = []
        try:
            specifiacija = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
            for line in specifiacija:
                try:
                    spec_key = line.find('span', {'class':'left-column'}).text
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('span', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
                try:
                    spec_key = line.find('div', {'class':'left-column'}).text
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('div', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
        except:
            pass

        try:
            specifiacija_k = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
            for key in specifiacija_k:
                # print(key.text.strip())
                specifiacija_keys.append(key.text.strip())
            specifiacija_v = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
            for val in specifiacija_v:
                # print(val.text.strip())
                specifiacija_values.append(val.text.strip())       
        except:
            pass
        
        
        # print(len(specifiacija_keys), specifiacija_keys)
        # print(len(specifiacija_values), specifiacija_values)

        spec_filtrai_keys = []
        spec_filtrai_values = []
        try:
            spec_filtai = bs_preke.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
            for line in spec_filtai:
                spec_key = line.find('span', {'class':'left-column'}).text.strip()
                # print(spec_key)
                spec_filtrai_keys.append(spec_key)
                spec_value = line.find('span', {'class':'right-column'}).text.strip()
                # print(spec_value)
                spec_filtrai_values.append(spec_value)
                
            # print(len(spec_filtrai_keys), spec_filtrai_keys)
            # print(len(spec_filtrai_values), spec_filtrai_values)
        except:
            pass
        
        for k, v in zip(spec_filtrai_keys, spec_filtrai_values):
            duomenys[k] = v
            
        for k, v in zip(specifiacija_keys, specifiacija_values):
            duomenys[k] = v
            
        cols_i_need = list(duomenys.keys())
        
        data = tuple(duomenys.values())
        
        cols_i_have = getColNamesList(table='DronaiVarle', dbname='VarlePigu.db')
        
        alterTableAddColumn(cols_i_need, cols_i_have, 'DronaiVarle', 'VarlePigu.db')
        
        writeMany(data, 'DronaiVarle', 'VarlePigu.db')
        print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
        counts = counts +1
    
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    if psl <= pages_max_number:
        next_nuoroda = f'https://www.varle.lt/dronai/?p={psl}'
        driver.get(next_nuoroda)
        time.sleep(3)
        print(f'kitas puslapis: {psl}')
    
driver.close()
print('+')


1 https://www.varle.lt/dronai/dji-mini-4-pro-fly-more-combo-dji-rc-2-dronas--29184156.html


OperationalError: no such table: DronaiVarle